In [ ]:
import numpy as np

# Load pointwise training set

Load the dataset generated [from the previous section](http://localhost:8888/notebooks/ch10/3.ch10-pairwise-transform.ipynb).

In [ ]:
pointwise_predictors = np.load('data/pointwise_predictors.npy')
feature_data = np.load('data/feature_data.npy')

std_devs = feature_data[-1]
means = feature_data[-2]
pointwise_features = feature_data[:-2]

## Listing 10.16

Train the model with the fully transformed dataset

In [ ]:
from sklearn import svm
model = svm.LinearSVC(max_iter=10000, verbose=1)
model.fit(pointwise_features, pointwise_predictors)
model.coef_

## Features for some other movies...

In [ ]:
# If you wanted to confirm Wrath of Khans features
import requests

logging_solr_query = {
    "fl": "id,title,[features store=movies efi.keywords=\"wrath of khan\"]",
    'q': "id:154", #social network graded documents
    'rows': 10,
    'wt': 'json'  
}

resp = requests.post('http://aips-solr:8983/solr/tmdb/select',
                     data=logging_solr_query)

# Features Solr returns
# Wrath of Khan
wok_features = [5.9217176, 3.401492, 1982.0]
# Search For Spock
spock_features = [0.0,0.0,1984.0]

# Wrath of Khan
normed_wok_features = [0,0,0]
for idx, f in enumerate(wok_features):
    normed_wok_features[idx] = (f - means[idx]) / std_devs[idx]

normed_spock_features = [0,0,0]
for idx, f in enumerate(spock_features):
    normed_spock_features[idx] = (f - means[idx]) / std_devs[idx]
    
normed_spock_features

## Score a single document

In [ ]:
def score_one(features, model):
    """ Stores one document (represented as feature array)
        using the provided model """
    score = 0.0
    print("Feature * Weight")
    for idx, f in enumerate(features):
        this_coef = np.asscalar(model.coef_[0][idx])
        score += f * this_coef

        # Explain for this feature
        print(" {:1.5f} * {:1.5f} ".format(f, this_coef))
        if (idx < len(features) - 1):
            print("+")
    
    print('= %s' % score)
    
    return score


score_one(normed_wok_features, model)